In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
from PIL import Image
import random
import csv

In [2]:
path = "/Volumes/Work/Mihir's Data/Machine Learning/COVID Classification/"
infectedImg_path = path + 'COVID'
healthyImg_path = path + 'non-COVID'

In [3]:
#Creating a CSV file since this dataset doesn't have one
fields = ['ImgName', 'Label']
filename = 'dataset.csv'
filepath = os.path.join(path, filename)
data = []
for i in os.listdir(infectedImg_path):
    if i[0] != '.':
        data.append([i, 1])

for i in os.listdir(healthyImg_path):
    if i[0] != '.':
        data.append([i, 0])

random.shuffle(data)

with open(filepath, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(data)

FileNotFoundError: [Errno 2] No such file or directory: "/Volumes/Work/Mihir's Data/Machine Learning/COVID Classification/COVID"

In [ ]:
len(data)

In [ ]:
#Loading the dataset
df = pd.read_csv(filepath)

#Splitting it into test and train dataset
df['split'] = np.random.randn(df.shape[0], 1)
msk = np.random.rand(len(df)) <= 0.8

train_df = df[msk]
test_df = df[~msk]

In [ ]:
#Creating the container object for images and labels

class COVIDClassificationDataset(Dataset):
    def __init__(self, dataFrame, root_dir, transform = None):
        self.dataFrame = dataFrame
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.dataFrame)
    
    def __getitem__(self, idx):
        label = self.dataFrame.iloc[idx, 1]
        if label == 0:
            newDir = os.path.join(self.root_dir, 'non-COVID')
        elif label == 1:
            newDir = os.path.join(self.root_dir, 'COVID')
        img_name = os.path.join(newDir, self.dataFrame.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return (image, label)

In [ ]:
img_transform = transforms.Compose([
    transforms.Resize(250),
    transforms.CenterCrop(227),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),


])

In [ ]:
covid_train = COVIDClassificationDataset(dataFrame=train_df, root_dir= path, transform=img_transform)
covid_test = COVIDClassificationDataset(dataFrame=test_df, root_dir=path, transform=img_transform)

In [ ]:
#Visualizing the image
import matplotlib.pyplot as plt

temp_img, temp_lab = covid_train[0]
plt.imshow(temp_img.numpy().transpose((1, 2, 0)))
if temp_lab:
    plt.title("Infected with COVID")
else:
    plt.title("Healthy")
plt.axis('off')
plt.show()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
#Dataloader
train_loader = DataLoader(dataset=covid_train, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=covid_test, batch_size=16, shuffle=False)

In [ ]:
#Transfer learning
model = torchvision.models.densenet121(pretrained=False)

In [ ]:
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 500),
    nn.Linear(500, 2)
)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, amsgrad=True)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)

In [ ]:
num_ep = 1
n_steps = len(train_loader)
model.train()
for ep in range(num_ep):
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        #Forward Pass
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)

        #Backward Pass
        loss.backward()
        optimizer.step()
    
    print(f'Epoch: [{ep + 1} / {num_ep}], Loss: {loss.item():.4f}')

In [ ]:
model.eval()
fn_list = []
pred_list = []
for x, fn in test_loader:
    with torch.no_grad():
        x = x.to(device)
        output = model(x)
        pred = torch.argmax(output, dim = 1)
        fn_list += [n[:-4] for n in fn]
        pred_list += [p.item() for p in pred]

print(pred)
print(fn_list)
print(pred_list)